In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "MyStrongPassword123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True, errors='ignore')

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# I included the Grazioso Salvare logo as required by the project
# instructions. The image is encoded in base64 so it can be
# embedded directly into the Dash layout.
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
     #    html.Div(id='hidden-div', style={'display':'none'}),
     # Dashboard title
    html.Center(html.B(html.H1('CS-340 Grazioso Salvare Dashboard'))),

    # Grazioso Salvare logo and unique identifier
    html.Center(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '100px'}
        )
    ),
    html.Center(html.H4('Stevan Ledesma– Project Two')),

    html.Hr(),

    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster / Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset (Show All)', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Hr(),

    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read_water_rescue())
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read_mountain_rescue())
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read_disaster_tracking())
    else:
        df = pd.DataFrame.from_records(db.read({}))

    df.drop(columns=['_id'], inplace=True, errors='ignore')
    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display.")

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution of Available Rescue Dogs')
    return dcc.Graph(figure=fig)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    dff = pd.DataFrame.from_dict(viewData)

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]


# Run app and display result in jupyterlab mode
app.run_server(mode='inline')


Successfully connected to MongoDB
Query successful. Found 10004 documents


Query successful. Found 10004 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
Query successful. Found 5 documents
Query successful. Found 18 documents
Query successful. Found 5 documents
Query successful. Found 18 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
